# Bước 1: thêm các thư viện và định nghĩa các hàm cần thiết

In [1]:
import pandas as pd
import numpy as np
import re
from fuzzywuzzy import process

In [2]:
# hàm để làm sạch cột trước khi xử lý
def clean_dataframe(df):
    """
    Ép toàn bộ các cột trong DataFrame sang kiểu chuỗi 
    và xoá khoảng trống ở đầu và cuối của các chuỗi.
    
    Parameters:
    df (pd.DataFrame): DataFrame cần làm sạch.

    Returns:
    pd.DataFrame: DataFrame đã được làm sạch.
    """
    # Xoá khoảng trắng đầu tên cột
    df.columns = df.columns.str.strip()
    
    # Ép toàn bộ DataFrame sang kiểu chuỗi
    df = df.astype(str)
    
    # Xoá khoảng trống đầu và cuối trong toàn bộ DataFrame
    df = df.applymap(lambda x: x.strip())
    
    return df

In [3]:
# Hàm xử lý chuỗi để sửa lỗi và giữ lại giá trị ngày
def clean_date(value):
    if not isinstance(value, str):  # Đảm bảo giá trị là chuỗi
        value = str(value)
    # Tìm chuỗi ngày trong dữ liệu
    match = re.search(r"\d{1,2}/\d{1,2}/\d{4}", value)
    return match.group(0) if match else value.strip()

In [4]:
#  Lấy danh sách tỉnh thành chuẩn và dùng thư viện fuzzywuzzy để sửa lại cột nơi sinh chính xác
def correct_tinh_thanh(tinh_thanh):
    # Kiểm tra nếu giá trị là chuỗi hợp lệ
    if isinstance(tinh_thanh, str):
        # Tìm kiếm tỉnh thành gần đúng nhất trong danh sách chuẩn
        match, score = process.extractOne(tinh_thanh, tinhthanh_chuan)
        return match if score >= 70 else "Không xác định"  # Chỉ sửa nếu độ chính xác >= 80%
    else:
        return "Không xác định"  # Trả về "Không xác định" nếu không phải là chuỗi

In [5]:
#  lấy điểm trung bình 5 học kỳ đầu
def pivot_top_5_semesters_diemtb_hk(df):
    """
    Chuyển đổi dữ liệu sao cho mỗi sinh viên là 1 dòng, với tối đa 5 học kỳ được xoay ngang.
    """
   # Sắp xếp dữ liệu theo mssv, namhoc, và hocky
    df_sorted = df.sort_values(by=['mssv', 'namhoc', 'hocky'])

    df_sorted = df_sorted.dropna()
    
    # Thêm cột số thứ tự học kỳ cho mỗi sinh viên
    df_sorted['hoc_ky_thu'] = df_sorted.groupby('mssv').cumcount() + 1
    
    # Lọc ra 5 học kỳ đầu tiên của mỗi sinh viên
    df_top_5 = df_sorted[df_sorted['hoc_ky_thu'] <= 5]
    
    # Pivot dữ liệu: mỗi học kỳ là một nhóm cột cho dtbhk và sotchk
    pivot_df = df_top_5.pivot(
        index='mssv',
        columns='hoc_ky_thu',
        values=['dtbhk', 'sotchk']
    )
    
    # Làm phẳng cột đa cấp và đổi tên cột theo định dạng yêu cầu
    pivot_df.columns = [f"{col[0]}_hk_{col[1]}" for col in pivot_df.columns]  # Đổi tên cột
    pivot_df.columns = [col.replace('dtbhk', 'dtbhk') if 'dtbhk' in col else col.replace('sotchk', 'sotchk') for col in pivot_df.columns]
    
    # Reset index để đưa mssv về thành cột
    pivot_df = pivot_df.reset_index()
    
    return pivot_df

In [6]:
#  lấy điểm rèn luyện 5 học kỳ đầu
def pivot_top_5_semesters_diemrl(df):
    """
    Chuyển đổi dữ liệu 'diemrl' sao cho mỗi sinh viên là 1 dòng, với tối đa 5 học kỳ được xoay ngang.
    """
    # Sắp xếp dữ liệu theo mssv, namhoc, và hocky
    df_sorted = df.sort_values(by=['mssv', 'namhoc', 'hocky'])

    df_sorted = df_sorted.dropna()
    
    # Thêm cột số thứ tự học kỳ cho mỗi sinh viên
    df_sorted['hoc_ky_thu'] = df_sorted.groupby('mssv').cumcount() + 1
    
    # Lọc ra 5 học kỳ đầu tiên của mỗi sinh viên
    df_top_5 = df_sorted[df_sorted['hoc_ky_thu'] <= 5]
    
    # Pivot dữ liệu: mỗi học kỳ là một nhóm cột chỉ cho điểm rèn luyện
    pivot_df = df_top_5.pivot(
        index='mssv',
        columns='hoc_ky_thu',
        values='drl'  # Chỉ lấy cột 'drl'
    )
    
    # Làm phẳng cột đa cấp với tên cột là drl_hk_x
    pivot_df.columns = [f"drl_hk_{col}" for col in pivot_df.columns]
    
    # Reset index để đưa mssv về thành cột
    pivot_df = pivot_df.reset_index()
    
    return pivot_df

# Step 2: Xử lý các bảng

# 1. totnghiep

In [7]:
# đọc bảng tốt nghiệp
totnghiep = pd.read_excel('Education_dataset_V2/14.totnghiep.xlsx')

In [8]:
totnghiep = clean_dataframe(totnghiep)

C:\Users\anhthy\AppData\Local\Temp\ipykernel_31368\2883437664.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip())


In [9]:
totnghiep.columns

Index(['id', 'mssv', 'xeploai', 'soquyetdinh', 'ngaycapvb'], dtype='object')

In [10]:
totnghiep.shape

(1847, 5)

In [11]:
# chỉ lấy 3 cột bên dưới
totnghiep = totnghiep [['mssv', 'xeploai', 'ngaycapvb']]

In [12]:
totnghiep

,mssv,xeploai,ngaycapvb
0,E95E7C6DXPvAibaEXe+1j/AqdkpM22DHf6P99fDJ,Khá,14/04/2017
1,D0FE4969XPvAibaEXe/yXiKgsgy0slCmJ5EKt6Ki,Giỏi,06/10/2017
2,40F7E8D0XPvAibaEXe+nZBq3b0XEhfcwXLmoc4Pj,Giỏi,06/10/2017
3,0A049F45XPvAibaEXe9CtFSNMbfIz9qE7i0Fu4My,Khá,06/10/2017
4,7A7166DAXPvAibaEXe83V/kFvw2bBZ0o/KVHZc30,Khá,06/10/2017
...,...,...,...
1842,A2698E63XPvAibaEXe+D5C9Re7EGLdYoD7VxJkyw,Khá,09/6/2021
1843,B44C3B36XPvAibaEXe/ns/wSuHF6kEcjw3s5q9gL,Khá,09/6/2021
1844,175475BBXPvAibaEXe8dBqMaN7rGhkux+ZR35bi9,Khá,09/6/2021
1845,94A79F45XPvAibaEXe+dJypLAXKUXx9tmvinQDNg,Khá,09/6/2021


In [13]:
totnghiep.columns

Index(['mssv', 'xeploai', 'ngaycapvb'], dtype='object')

In [14]:
totnghiep['xeploai'].value_counts()

xeploai
Khá               1125
Giỏi               447
Trung bình khá     132
TB Khá              92
Trung bình Khá      40
Xuất sắc             7
nan                  2
TB khá               2
Name: count, dtype: int64

In [15]:
totnghiep['xeploai'] = totnghiep['xeploai'].replace('TB Khá', 'Trung bình khá')
totnghiep['xeploai'] = totnghiep['xeploai'].replace('Trung bình Khá', 'Trung bình khá')
totnghiep['xeploai'] = totnghiep['xeploai'].replace('TB khá', 'Trung bình khá')

In [16]:
totnghiep['xeploai'].value_counts()

xeploai
Khá               1125
Giỏi               447
Trung bình khá     266
Xuất sắc             7
nan                  2
Name: count, dtype: int64

In [17]:
# Vì định dạng ngày có thể không nhất quán, dùng hàm để tự động thay thế
totnghiep['ngaycapvb'] = pd.to_datetime(totnghiep['ngaycapvb'], errors='coerce', dayfirst=True)
totnghiep['ngaycapvb'] = totnghiep['ngaycapvb'].dt.strftime('%d/%m/%Y')

In [18]:
totnghiep.isna().sum()

mssv         0
xeploai      0
ngaycapvb    6
dtype: int64

In [19]:
totnghiep = totnghiep.dropna()

In [20]:
totnghiep.isna().sum()

mssv         0
xeploai      0
ngaycapvb    0
dtype: int64

# 2. sinhvien

In [21]:
sinhvien = pd.read_excel('Education_dataset_V2/01.sinhvien.xlsx')

In [22]:
sinhvien = clean_dataframe(sinhvien)

C:\Users\anhthy\AppData\Local\Temp\ipykernel_31368\2883437664.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip())


In [23]:
sinhvien.head(4)

,id,mssv,namsinh,gioitinh,noisinh,lopsh,khoa,hedt,khoahoc,chuyennganh2,...,_47,_48,_49,_50,_51,_52,_53,_54,_55,_56
0,14911,E37BF33CXPvAibaEXe9Y6XKwco8jyUy8xdOxIJZW,1999.0,1,'Hải Dương',MMTT2017,MMT&TT,CQUI,12,D480102,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,18120,B3024A0DXPvAibaEXe++B0JyCbXA1F/t+O6kpXJd,2001.0,1,'Bình Phước',HTCL2019.1,HTTT,CLC,14,D480104,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,9278,0ACD9F7AXPvAibaEXe/K6d36/fqcEaBjFfA/Uaef,1994.0,1,'TP. Hồ Chí Minh',KHTN2013,KHMT,CNTN,8,D480101,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,19024,700E1725XPvAibaEXe+SbxsvF9MWxKMQLP/zEM+u,2001.0,1,'An Giang',KHCL2019.3,KHMT,CLC,14,D480101,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [24]:
sinhvien.columns

Index(['id', 'mssv', 'namsinh', 'gioitinh', 'noisinh', 'lopsh', 'khoa', 'hedt',
       'khoahoc', 'chuyennganh2', 'tinhtrang', 'diachi_tinhtp', 'Column1',
       '_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11',
       '_12', '_13', '_14', '_15', '_16', '_17', '_18', '_19', '_20', '_21',
       '_22', '_23', '_24', '_25', '_26', '_27', '_28', '_29', '_30', '_31',
       '_32', '_33', '_34', '_35', '_36', '_37', '_38', '_39', '_40', '_41',
       '_42', '_43', '_44', '_45', '_46', '_47', '_48', '_49', '_50', '_51',
       '_52', '_53', '_54', '_55', '_56'],
      dtype='object')

In [25]:
# lấy các cột có giá trị sử dụng
sinhvien = sinhvien[['mssv', 'namsinh', 'noisinh', 'gioitinh', 'khoa', 'hedt',
       'khoahoc', 'tinhtrang']]

In [26]:
sinhvien.head(20)

,mssv,namsinh,noisinh,gioitinh,khoa,hedt,khoahoc,tinhtrang
0,E37BF33CXPvAibaEXe9Y6XKwco8jyUy8xdOxIJZW,1999.0,'Hải Dương',1,MMT&TT,CQUI,12,5
1,B3024A0DXPvAibaEXe++B0JyCbXA1F/t+O6kpXJd,2001.0,'Bình Phước',1,HTTT,CLC,14,5
2,0ACD9F7AXPvAibaEXe/K6d36/fqcEaBjFfA/Uaef,1994.0,'TP. Hồ Chí Minh',1,KHMT,CNTN,8,5
3,700E1725XPvAibaEXe+SbxsvF9MWxKMQLP/zEM+u,2001.0,'An Giang',1,KHMT,CLC,14,1
4,2B578231XPvAibaEXe9RV3kg6cpLDAmMozUvJMwR,1997.0,'Đồng Nai',0,MMT&TT,CQUI,10,5
5,7806D9BAXPvAibaEXe9IFJLLU9DVVQ7bFL8KvJIW,1998.0,'Đồng Nai',1,MMT&TT,CQUI,11,1
6,B1EA4A9EXPvAibaEXe8oT/p9f7HpvUf1MQPdsSuF,2000.0,'Đồng Nai',1,CNPM,CLC,13,1
7,1F8B9DB8XPvAibaEXe9cUsS4F6r57XT3g2N1sVl6,2001.0,'Đồng Nai',0,HTTT,CQUI,14,1
8,84AF1CF1XPvAibaEXe9A8avg2Xms/yHRCW9PwY1i,2000.0,'Lâm Đồng',0,HTTT,CQUI,13,1
9,B327FD18XPvAibaEXe/VQHzzHDYACaWUicUcibVi,2001.0,'Đồng Nai',0,MMT&TT,CQUI,14,1


In [27]:
sinhvien['khoahoc'].value_counts()

khoahoc
14    1718
13    1424
12    1089
10    1042
11    1041
9     1009
8      972
Name: count, dtype: int64

xử lý địa chỉ của sinh viên

In [28]:
#  lấy dữ liệu tỉnh thành chuẩn
tinhthanh_chuan = pd.read_excel('Education_dataset_V2/tỉnh thành Việt Nam cập nhật đến 2021.xlsx')
tinhthanh_chuan = tinhthanh_chuan['Tỉnh thành'].values.tolist()

In [29]:
# # Chuyển tất cả chữ cái trong tên cột thành chữ thường
# sinhvien['noisinh'] = sinhvien['noisinh'].str.lower()

# Xóa ký tự "'", Tỉnh, Thành phố
sinhvien['noisinh'] = sinhvien['noisinh'].str.replace("'", '', regex=False)
sinhvien['noisinh'] = sinhvien['noisinh'].str.replace('tỉnh', '', regex=False)
sinhvien['noisinh'] = sinhvien['noisinh'].str.replace('thành phố', '', regex=False)

# Thay thế n/a bằng không xác định
sinhvien['noisinh'] = sinhvien['noisinh'].fillna('Không xác định').astype(str)

In [30]:
sinhvien['noisinh'] = sinhvien['noisinh'].apply(correct_tinh_thanh)

In [31]:
sinhvien['noisinh'].value_counts()

noisinh
Hồ Chí Minh    1629
Đồng Nai        525
Đắk Lắk         501
Bình Định       414
Lâm Đồng        354
               ... 
Thái Nguyên       3
Lai Châu          2
Tuyên Quang       1
Bắc Kạn           1
Cao Bằng          1
Name: count, Length: 61, dtype: int64

In [32]:
sinhvien.shape

(8295, 8)

In [33]:
sinhvien.isna().sum()

mssv         0
namsinh      0
noisinh      0
gioitinh     0
khoa         0
hedt         0
khoahoc      0
tinhtrang    0
dtype: int64

In [34]:
sinhvien['tinhtrang'].value_counts()

tinhtrang
1     4342
3     1845
5     1318
8      374
2      302
4       82
11      22
6       10
Name: count, dtype: int64

In [35]:
sinhvien = sinhvien.dropna()

In [36]:
# Thay thế giới tính từ id thành string với 1 là nam, 0 là nữ
sinhvien['gioitinh'] = sinhvien['gioitinh'].str.replace("1", 'Nam', regex=False)
sinhvien['gioitinh'] = sinhvien['gioitinh'].str.replace('0', 'Nữ', regex=False)

# 3. xeploaiav

In [37]:
xeploaiav = pd.read_excel('Education_dataset_V2/04.xeploaiav.xlsx')

In [38]:
xeploaiav = clean_dataframe(xeploaiav)

C:\Users\anhthy\AppData\Local\Temp\ipykernel_31368\2883437664.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip())


In [39]:
xeploaiav.shape

(6349, 8)

In [40]:
xeploaiav.columns

Index(['id', 'mssv', 'listening', 'reading', 'Column1', 'total', 'mamh',
       'ghichu'],
      dtype='object')

In [41]:
#  lấy các cột có giá trị sử dụng
xeploaiav = xeploaiav [['mssv', 'mamh']]

In [42]:
# Loại bỏ khoảng trắng đầu và cuối 
xeploaiav['mamh'] = xeploaiav['mamh'].str.strip()

# Xóa ký tự "'" trong mamh
xeploaiav['mamh'] = xeploaiav['mamh'].str.replace("'", '', regex=False)

# Thay thế EN001 bằn ENG01
xeploaiav['mamh'] = xeploaiav['mamh'].replace('EN001', 'ENG01')
# Thay thế EN002 bằn ENG02
xeploaiav['mamh'] = xeploaiav['mamh'].replace('EN002', 'ENG02')

In [43]:
#  chỉ lấy các dùng có giá trị đúng (bỏ qua các giá trị như NULL, N/A)
xeploaiav = xeploaiav[(xeploaiav['mamh'].isin(['ENG01', 'AVSC1', 'AVSC2', 'ENG02', 'AVSC', 'ENG04', 'ENG05', 'Miễn ENG03']))]

In [44]:
xeploaiav['mamh'].value_counts()

mamh
ENG01         1860
AVSC1         1261
AVSC2          824
ENG02          785
AVSC           415
ENG04          261
ENG05          225
Miễn ENG03     111
Name: count, dtype: int64

In [45]:
xeploaiav.shape

(5742, 2)

In [46]:
xeploaiav.isna().sum()

mssv    0
mamh    0
dtype: int64

In [ ]:
# Vì bảng xeploaiav không bao gồm chuẩn đầu vào tiếng anh cho khoá 8 9 10 nên dùng điểm thi để xác định mamh anh văn đầu vào
diem_Thu = pd.read_excel('Education_dataset_V2/diem_Thu.xlsx')

In [ ]:
#  làm sạch điểm thu
diem_Thu = clean_dataframe(diem_Thu)

In [ ]:
diem_Thu.columns

In [ ]:
# diem_Thu = diem_Thu[['mssv', 'mamh']]
diem_Thu = diem_Thu[(diem_Thu['mamh'] == 'ENBT') | (diem_Thu['mamh'] == 'ADENG01') | (diem_Thu['mamh'] == 'ADENG02') |
    (diem_Thu['mamh'] == 'ADENG02') | (diem_Thu['mamh'] == 'ADENG03') | (diem_Thu['mamh'] == 'ENGL1113') | (diem_Thu['mamh'] == 'ENGL1213') ]

In [ ]:
# diem_Thu = diem_Thu[['mssv', 'mamh']]
diem_Thu = diem_Thu[(diem_Thu['mamh'] == 'ENBT') | (diem_Thu['mamh'] == 'ADENG01') | (diem_Thu['mamh'] == 'ADENG02') |
    (diem_Thu['mamh'] == 'ADENG02') | (diem_Thu['mamh'] == 'ADENG03') | (diem_Thu['mamh'] == 'ENGL1113') | (diem_Thu['mamh'] == 'ENGL1213') ]

#  lấy các cột có giá trị sử dụng từ diem_Thu và sắp xếp để sau đó lấy môn đầu tiên tức là môn av đầu vào
diem_Thu = diem_Thu.sort_values(by=['mssv', 'namhoc', 'hocky'])

# Lấy mã môn đầu vào của sinh viên (lấy mã đầu tiên - av đầu vào) 
diem_Thu = diem_Thu.groupby('mssv').first().reset_index()

# Nếu chỉ lấy cột 'mssv' và 'mamh'
diem_Thu = diem_Thu[['mssv', 'mamh']]

# 4. diemrl

In [ ]:
diemrl = pd.read_excel('Education_dataset_V2/diemrl.xlsx')

In [ ]:
diemrl = clean_dataframe(diemrl)

In [ ]:
diemrl.columns

In [ ]:
#  đếm số lượng
diemrl['drl'].value_counts()

In [ ]:
diemrl

In [ ]:
diemrl.columns

In [ ]:
#  lấy các cột có giấ trị sử dụng
diemrl = diemrl[['mssv', 'hocky', 'namhoc', 'drl']]

In [ ]:
#  lấy 5 học kỳ đầu
diemrl = pivot_top_5_semesters_diemrl(diemrl)

In [ ]:
diemrl

# 5. thisinh

In [ ]:
thisinh = pd.read_excel('Education_dataset_V2/05.ThiSinh.xlsx')

In [ ]:
thisinh = clean_dataframe(thisinh)

In [ ]:
thisinh.columns

In [ ]:
thisinh['dien_tt'].value_counts()

In [ ]:
thisinh

In [ ]:
thisinh.columns

In [ ]:
thisinh = thisinh[['mssv', 'dien_tt', 'diem_tt', 'lop12_matinh']]

# 6. Diem

In [ ]:
diemtb_hk = pd.read_excel('Education_dataset_V2/sinhvien_dtb_hocky.xlsx')

In [ ]:
diemtb_hk = clean_dataframe(diemtb_hk)

In [ ]:
diemtb_hk.columns

In [ ]:
# lấy 5 học kỳ đầu
diemtb_hk = pivot_top_5_semesters_diemtb_hk(diemtb_hk)

In [ ]:
diemtb_hk

# 7 Xử lý học vụ

In [ ]:
xlhv = pd.read_excel('Education_dataset_V2/08.XLHV.xlsx')
xlhv = clean_dataframe(xlhv)

In [ ]:
xlhv.columns

In [ ]:
# Vì định dạng ngày có thể không nhất quán, dùng hàm để tự động thay thế
xlhv['ngayqd'] = pd.to_datetime(xlhv['ngayqd'], errors='coerce', dayfirst=True)
xlhv['ngayqd'] = xlhv['ngayqd'].dt.strftime('%d/%m/%Y')

In [ ]:
xlhv

In [ ]:
xlhv = xlhv[['id', 'mssv', 'tinhtrang', 'lydo', 'hocky', 'namhoc']]

# Bước 3: tạo bảng fact

In [ ]:
fact = None

In [ ]:
totnghiep

In [ ]:
#  Bảng XLHV chỉ lấy được các sinh viên không tốt nghiệp được và gánh là khong tốt nghiệp
# trong bảng xlhv lấy các sinh viên có tình trạng là 5 7 8 tức không hoàn thành chương trình để tốt nghiệp
xlhv = xlhv[(xlhv['tinhtrang'] == 5) | (xlhv['tinhtrang'] == 7) | (xlhv['tinhtrang'] == 8)]

# Gán giá tất cả thành không được tốt nghiệp
xlhv['tinhtrang'] = "Không tốt nghiệp"

# Đổi tên tinhtrang thành xeploai và ngayqd thành ngaycapvb (sẽ đổi tên trong bảng fact sau) để ghép với bảng totnghiep
xlhv = xlhv.rename(columns={'tinhtrang': 'xeploai', 'ngayqd': 'ngaycapvb'})

In [ ]:
xlhv.columns

In [ ]:
# lấy các cột có giá trị
xlhv = xlhv[['mssv', 'xeploai']]

In [ ]:
xlhv.isna().sum()

In [ ]:
xlhv = xlhv.dropna()

In [ ]:
fact = pd.concat([totnghiep, xlhv], ignore_index=True)

In [ ]:
fact

In [ ]:
sinhvien

In [ ]:
# gộp bảng sinhvien vào fact
fact = pd.merge(fact, sinhvien, how='left', on='mssv')

In [ ]:
fact

In [ ]:
fact.isna().sum()

In [ ]:
# Gộp bảng 'xeploaiav' vào 'fact'
fact = pd.merge(fact, xeploaiav, how='left', on='mssv', suffixes=('', '_drop'))

# Loại bỏ các cột không cần thiết (nếu trùng tên)
fact = fact.loc[:, ~fact.columns.str.endswith('_drop')]

# Gộp tiếp bảng 'diem_Thu' chỉ cho các hàng còn thiếu giá trị
fact = pd.merge(fact, diem_Thu, how='left', on='mssv', suffixes=('', '_drop'))

# Ưu tiên dữ liệu từ 'xeploaiav', điền dữ liệu từ 'diem_Thu' cho các hàng còn thiếu
for col in diem_Thu.columns:
    if col != 'mssv':  # Bỏ qua khóa chính
        fact[col] = fact[col].combine_first(fact[f"{col}_drop"])

# Loại bỏ các cột thừa sau khi gộp
fact = fact.loc[:, ~fact.columns.str.endswith('_drop')]

In [ ]:
# fact

In [ ]:
fact.isna().sum()

In [ ]:
# gộp bảng thisinh vào fact
fact = pd.merge(fact, thisinh, how='left', on='mssv')


In [ ]:
fact.shape

In [ ]:
# gộp bảng diemtb_hk vào fact
fact = pd.merge(fact, diemtb_hk, how='left', on='mssv')

In [ ]:
fact.isna().sum()

In [ ]:
diemrl

In [ ]:
# gộp bảng diemrl vào fact
fact = pd.merge(fact, diemrl, how='left', on='mssv')

In [ ]:
fact.isna().sum()

In [ ]:
fact.shape

In [ ]:
fact.isna().sum()

In [ ]:
# xoá khoá 12 vì dữ liệu bị ít
fact = fact[(fact['khoahoc'] != '12')]

In [ ]:
# tính toán khoá chuẩn của sinh viên ví dụ sinh năm 2003 thì khoá học chuẩn là k16 bằng cách lấy năm sinh trừ đi 1987
# Chuyển cột 'namsinh' thành kiểu số
fact['namsinh'] = pd.to_numeric(fact['namsinh'], errors='coerce')  # Chuyển đổi giá trị không hợp lệ thành NaN
fact.dropna(subset=['namsinh'], inplace=True)  # Loại bỏ hàng có NaN trong 'namsinh'
fact['namsinh'] = fact['namsinh'].astype(float).astype(int)  # Loại bỏ phần thập phân và chuyển thành int

# Tính toán khóa học chuẩn
fact['khoahoc_chuan'] = fact['namsinh'] - 1987

In [ ]:
fact.shape

In [ ]:
#  xoá các cột không cần thiết
columns_to_drop = ['mamh', 'lop12_matinh', 'dien_tt', 'ngaycapvb', 'tinhtrang' ]
fact = fact.drop(columns=columns_to_drop)

In [ ]:
fact = fact.dropna()

# Ép kiểu các loại dữ liệu sang kiểu phù hợp

In [ ]:
# Ép kiểu các cột liên quan đến MSSV, xếp loại, nơi sinh, giới tính, khoa, hệ đào tạo, khóa học, tình trạng, và mã tỉnh lớp 12 thành string
columns_to_string = [
    'mssv', 'xeploai', 'noisinh', 'gioitinh', 
    'khoa', 'hedt', 'khoahoc'
]

fact[columns_to_string] = fact[columns_to_string].astype(str)

# Ép kiểu các cột điểm thành float
columns_to_float = [
    'dtbhk_hk_1', 'dtbhk_hk_2', 'dtbhk_hk_3', 
    'dtbhk_hk_4', 'dtbhk_hk_5', 
    'drl_hk_1', 'drl_hk_2', 'drl_hk_3', 'drl_hk_4', 'drl_hk_5'
]
fact[columns_to_float] = fact[columns_to_float].astype(float)

# Ép kiểu các cột số tín chỉ và năm sinh thành int
columns_to_int = [
    'namsinh', 'sotchk_hk_1', 'sotchk_hk_2', 
    'sotchk_hk_3', 'sotchk_hk_4', 'sotchk_hk_5', 'khoahoc_chuan'
]
fact[columns_to_int] = fact[columns_to_int].astype(int)

In [ ]:
fact.shape

In [ ]:
fact['diem_tt'].value_counts()

In [ ]:
fact = fact[fact['diem_tt'] != 'nan']

fact = fact[fact['diem_tt'] != '0.0']

In [ ]:
fact.shape

In [ ]:
fact.to_excel("..\store\\fact.xlsx", index=False)